In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter, Venus
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат) 1762 дней полета
date_launch = Time("2026-07-09", scale="utc").tdb
date_flyby_Earth = Time("2029-04-11", scale="utc").tdb
date_flyby_Venus = Time("2030-02-03", scale="utc").tdb
date_arrival = Time("2034-05-07", scale="utc").tdb

In [3]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
ven = Ephem.from_body(Venus, time_range(date_launch, end=date_flyby_Venus))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [4]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_ven = Orbit.from_ephem(Sun, ven, date_flyby_Venus)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [5]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_earth_2)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_earth_2, ss_ven)
ss_trans_2, ss_target_2 = ss_earth_2.apply_maneuver(man_lambert_2, intermediate=True)

man_lambert_3 = Maneuver.lambert(ss_ven, ss_jup)
ss_trans_3, ss_target_3 = ss_ven.apply_maneuver(man_lambert_3, intermediate=True)

In [41]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=320 * u.deg), color="blue", label="Transfer orbit E-E"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=70 * u.deg), color="purple", label="Transfer orbit E-V"
)

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=198 * u.deg), color="black", label="Transfer orbit V-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [7]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth_1")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=320 * u.deg), color="blue", label="Transfer orbit E-M"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth_2")
plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=70 * u.deg), color="purple", label="Transfer orbit M-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [42]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(ven, date_flyby_Venus, label="Venus")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_3.sample(max_anomaly=198 * u.deg), color="black", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [10]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_ven, v_ven = Ephem.from_body(Venus, date_flyby_Venus).rv(date_flyby_Venus)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [11]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

42834998.79830547 km
-134556471.14523274 km
-58311296.31814858 km

[28.10263038  7.63768945  3.30983355] km / s


In [12]:
# Фазовые координаты Марса

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-139796078.26578867 km
-49379812.46847284 km
-21399348.47494507 km

[ 10.17964763 -25.62317567 -11.10739398] km / s


In [13]:
# Фазовые координаты Венеры

print (f"{r_ven [0]}")
print (f"{r_ven [1]}")
print (f"{r_ven [2]}"'\n')

print (v_ven.to(u.km / u.s))

-93287698.87472689 km
46242698.90286343 km
26720000.397357162 km

[-17.43994655 -28.32488771 -11.64281276] km / s


In [14]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

738967637.7226346 km
-54512635.34936055 km
-41349052.75078177 km

[ 1.00664594 12.53152119  5.3467543 ] km / s


In [15]:
nh_tof_1 = date_flyby_Earth - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_earth), = iod.lambert(Sun.k, r_earth, r_earth_2, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=1007.0000000201045>

In [16]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 11.2560661 km / s>

In [17]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [ 8.01427206, -7.25215503, -3.1427241 ] km / s>

In [18]:
# Гиперболический избыток скорости при подлете к Земле

V_inf_end = (norm(vk_earth - v_earth_2)).to(u.km / u.s)
V_inf_end

<Quantity 11.11259806 km / s>

In [19]:
# Гиперболический избыток скорости при подлете к Марсу

V_inf_end_vec = (vk_earth - v_earth_2).to(u.km / u.s)
V_inf_end_vec

<Quantity [10.93857264, -1.79864901, -0.7760966 ] km / s>

In [20]:
nh_tof_2 = date_flyby_Venus - date_flyby_Earth # время полета Земля - Венера

(v0_earth, vk_ven), = iod.lambert(Sun.k, r_earth_2, r_ven, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=297.99999999084946>

In [21]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_end_0 = (norm(v0_earth - v_earth_2)).to(u.km / u.s)
V_inf_end_0

<Quantity 10.84573014 km / s>

In [22]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_0_vec = (v0_earth - v_earth_2).to(u.km / u.s)
V_inf_end_0_vec

<Quantity [-10.61877733,  -0.11431164,  -2.2041695 ] km / s>

In [23]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_1 = V_inf_end - V_inf_end_0
v_gip_diff_1

<Quantity 0.26686791 km / s>

In [24]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2 = (norm(vk_ven - v_ven)).to(u.km / u.s)
V_inf_end_2

<Quantity 11.89693237 km / s>

In [25]:
# Гиперболический избыток скорости при подлете к Венере

V_inf_end_2_vec = (vk_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-11.14638139,   4.15871447,  -0.01660857] km / s>

In [26]:
nh_tof_3 = date_arrival - date_flyby_Venus # время полета Венера - Юпитер

(V0_ven, Vk_Jup), = iod.lambert(Sun.k, r_ven, r_jup, nh_tof_3)
nh_tof_3

<TimeDelta object: scale='tdb' format='jd' value=1554.000000006553>

In [27]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_0 = (norm(V0_ven - v_ven)).to(u.km / u.s)
V_inf_end_2_0

<Quantity 12.3670124 km / s>

In [28]:
# Гиперболический избыток скорости при отлете от Венеры

V_inf_end_2_vec_0 = (V0_ven - v_ven).to(u.km / u.s)
V_inf_end_2_vec_0

<Quantity [-3.65950225, -8.53917571, -8.16293557] km / s>

In [29]:
# Разница гиперболических избытков скорости подлета и отлета 

v_gip_diff_2 = V_inf_end_2 - V_inf_end_2_0
v_gip_diff_2

<Quantity -0.47008002 km / s>

In [30]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k = (norm(Vk_Jup - v_jup)).to(u.km / u.s)
V_inf_3_k

<Quantity 9.1895433 km / s>

In [31]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_3_k_vec = (Vk_Jup - v_jup).to(u.km / u.s)
V_inf_3_k_vec

<Quantity [-6.59950686, -6.14485356, -1.77059029] km / s>

In [32]:
time_adv = nh_tof_1 + nh_tof_2 + nh_tof_3 
time_adv

<TimeDelta object: scale='tdb' format='jd' value=2859.000000017507>

In [33]:
scalar_EE2 = r_earth[0]*r_earth_2[0] + r_earth[1]*r_earth_2[1] +r_earth[2]*r_earth_2[2] 
scalar_E2V = r_ven[0]*r_earth_2[0] + r_ven[1]*r_earth_2[1] +r_ven[2]*r_earth_2[2]
scalar_VJ = r_ven[0]*r_jup[0] + r_ven[1]*r_jup[1] +r_ven[2]*r_jup[2]

In [34]:
fi_EE2 = (scalar_EE2/norm(r_earth)/norm(r_earth_2))
fi_E2V = (scalar_E2V/norm(r_earth_2)/norm(r_ven))
fi_VJ = (scalar_VJ/norm(r_ven)/norm(r_jup))

In [35]:
np.degrees(np.arccos(fi_EE2))

<Quantity 85.22756331 deg>

In [36]:
np.degrees(np.arccos(fi_E2V))

<Quantity 50.75920566 deg>

In [37]:
np.degrees(np.arccos(fi_VJ))

<Quantity 155.44937691 deg>

In [38]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 11.256066 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 7.83886189 km / s>

In [39]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 11.137628

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

15.66422189447073
17.983782441612995


<Quantity -2.31956055 km / s>

In [40]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 44.3188547 * to_rad 
ist_an_2 = 318.7633549 * to_rad 

exs_EM = (norm(r_earth_2)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_earth_2)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.8915034354523979
Фокальный параметр 250222181.52803963 km
Большая полуось 1219277851.6606894 km


In [97]:
## параметры участка Марс-Венера 

ist_an_1_MV = 119.7987395 * to_rad  
ist_an_2_MV = 70.7300112 * to_rad  

exs_MV = (norm(r_ven)/norm(r_earth_2) - 1) / (cos(ist_an_1_MV) - (norm(r_ven)/norm(r_earth_2) * cos(ist_an_2_MV)))
p_MV = norm(r_earth_2)*(1+cos(ist_an_1_MV)*exs_MV)
a_MV = p_MV/(1 - exs_MV*exs_MV)

print (f"Эксцентриситет {exs_MV}")
print (f"Фокальный параметр {p_MV}")
print (f"Большая полуось {a_MV}")

Эксцентриситет 0.3848647121905905
Фокальный параметр 121147037.60924 km
Большая полуось 142211529.81435838 km


In [98]:
## параметры участка Венера-Юпитер

ist_an_1_VJ = 354.5888241 * to_rad   
ist_an_2_VJ = 197.6319704 * to_rad   

exs_VJ = (norm(r_jup)/norm(r_ven) - 1) / (cos(ist_an_1_VJ) - (norm(r_jup)/norm(r_ven) * cos(ist_an_2_VJ)))
p_VJ = norm(r_ven)*(1+cos(ist_an_1_VJ)*exs_VJ)
a_VJ = p_VJ/(1 - exs_VJ*exs_VJ)

print (f"Эксцентриситет {exs_VJ}")
print (f"Фокальный параметр {p_VJ}")
print (f"Большая полуось {a_VJ}")

Эксцентриситет 0.7793817155519431
Фокальный параметр 190899342.79533935 km
Большая полуось 486288284.21197325 km
